# 4.2 Develop the regression model

In [1]:
# for the sake of development, use this magic command to solve slow suggestion
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('max_columns', 50)
from load_data import load_data
from preprocess_data import preprocess

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, make_scorer

from sklearn.preprocessing import OneHotEncoder

#### Load in the dataset

In [4]:
df = preprocess(load_data())
df.head()

,revenue,runtime,num_prods,num_languages,num_writers,UNRATE,PCE,class,original_language_en,original_language_fr,original_language_hi,month_Apr,month_Aug,month_Dec,month_Feb,month_Jan,month_Jul,month_Jun,month_Mar,month_May,month_Nov,month_Oct,month_Sep,genres_Adventure,genres_Animation,...,prod_comp_names_Universal_Pictures,prod_comp_names_Columbia_Pictures,prod_comp_names_Paramount,prod_comp_names_20th_Century_Fox,prod_comp_names_New_Line_Cinema,prod_comp_names_Walt_Disney_Pictures,prod_comp_names_Canal+,prod_comp_names_Metro-Goldwyn-Mayer,prod_comp_names_Touchstone_Pictures,prod_comp_names_Relativity_Media,prod_comp_names_Miramax,prod_comp_cntry_US,prod_comp_cntry_GB,prod_comp_cntry_FR,num_top_100_actors,established_director,log10_budget,log10_director_pop,log10_avg_writer_pop,log10_max_writer_pop,log10_avg_actor_pop,log10_max_actor_pop,log10_min_actor_pop,log10_cast_crew_sum_pop,log10_cast_crew_product_pop
0,2.215460e+08,81.0,1,1,5,5.5,5013.9,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.0,7.250232,0.525304,0.652182,0.822822,1.051268,1.429510,0.393048,1.280904,2.228754
1,1.562650e+08,104.0,4,2,3,5.6,5097.5,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.0,7.587154,0.471732,0.260389,0.629817,0.876776,1.069668,0.318272,1.090399,1.608897
2,4.843322e+07,127.0,1,1,2,5.6,5097.5,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0.0,6.978361,0.725830,-0.175874,0.146438,0.524006,0.754578,0.199206,0.969789,1.073962
3,1.114540e+08,170.0,3,2,1,5.6,5097.5,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,1.0,7.552392,0.962985,0.485863,0.962985,1.037811,1.211307,0.914079,1.364620,2.486659
4,3.191459e+07,127.0,7,2,5,5.6,5097.5,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1.0,7.537669,0.031408,0.337858,0.350829,0.858918,1.054498,0.697142,1.020292,1.228185


## 1. Regression

Develop the regression model for predicting the revenue that a film will make. We will actually predict $log_{10}$ of the revenue, and then transform it back.

In [7]:
X = df.drop(['revenue', 'class'], axis=1)
y = df['revenue'].apply(lambda x: math.log10(x))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
X_train.head(3)

,runtime,num_prods,num_languages,num_writers,UNRATE,PCE,year,original_language_en,original_language_fr,original_language_hi,month_Apr,month_Aug,month_Dec,month_Feb,month_Jan,month_Jul,month_Jun,month_Mar,month_May,month_Nov,month_Oct,month_Sep,genres_Adventure,genres_Animation,genres_Children,...,prod_comp_names_Universal_Pictures,prod_comp_names_Columbia_Pictures,prod_comp_names_Paramount,prod_comp_names_20th_Century_Fox,prod_comp_names_New_Line_Cinema,prod_comp_names_Walt_Disney_Pictures,prod_comp_names_Canal+,prod_comp_names_Metro-Goldwyn-Mayer,prod_comp_names_Touchstone_Pictures,prod_comp_names_Relativity_Media,prod_comp_names_Miramax,prod_comp_cntry_US,prod_comp_cntry_GB,prod_comp_cntry_FR,num_top_100_actors,established_director,log10_budget,log10_director_pop,log10_avg_writer_pop,log10_max_writer_pop,log10_avg_actor_pop,log10_max_actor_pop,log10_min_actor_pop,log10_cast_crew_sum_pop,log10_cast_crew_product_pop
4647,92.0,3,1,1,5.9,11948.8,2014,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.0,6.740775,-0.010995,-0.698970,-0.221849,0.898890,1.167672,0.159266,0.958946,0.188924
4253,118.0,3,1,1,6.6,11503.7,2014,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0.0,8.037783,0.060698,-0.485895,-0.008774,0.989687,1.172311,0.836134,1.050844,0.564490
4681,145.0,2,1,4,4.4,9651.3,2007,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,6.943851,-0.221849,0.086004,0.040207,0.184313,0.283527,0.086360,0.524742,0.048468


In [10]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [9]:
lin_reg = LinearRegression()
svm_reg = SVR()
tree_reg = DecisionTreeRegressor()
forest_reg = RandomForestRegressor()

models = [lin_reg, svm_reg, tree_reg, forest_reg]

In [10]:
for model in models:
    model.fit(X_train, y_train)

In [11]:
X_test = scaler.transform(X_test)

In [12]:
for model in models:
    predictions = (model.predict(X_test))
    print(f'Model scoring: {type(model)}')
    print(f"r2-score = {r2_score(y_test, predictions)}")
    print(f"mse = {mean_squared_error(y_test, predictions)}")
    print(f"rmse = {math.sqrt(mean_squared_error(y_test, predictions))}")
    print()

Model scoring: <class 'sklearn.linear_model._base.LinearRegression'>
r2-score = 0.7840800679821973
mse = 0.15986047179410762
rmse = 0.3998255517023738

Model scoring: <class 'sklearn.svm._classes.SVR'>
r2-score = 0.7701181700559275
mse = 0.1701974312807873
rmse = 0.4125499136841351

Model scoring: <class 'sklearn.tree._classes.DecisionTreeRegressor'>
r2-score = 0.611227045224044
mse = 0.28783552954318886
rmse = 0.5365030564155147

Model scoring: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
r2-score = 0.7861148256343287
mse = 0.15835399985695656
rmse = 0.39793718079234136



In [20]:
print("${:.2f}".format(10 ** predictions[0]))
print("${:.2f}".format(10 ** y_test.iloc[0]))

$201811.85
$561067.54


Take Random Forest and Logistic Regression forward for further tuning.

In [15]:
param_grid = {
    'n_estimators': [100, 200, 500]
    , 'max_features': ['auto', 'sqrt', 'log2']
    , 'max_depth' : [100, 200, 500]
#     , 'splitter': ['best', 'random']
}

forest_reg = RandomForestRegressor()

r2 = make_scorer(r2_score)  

grid_search = GridSearchCV(forest_reg, param_grid, cv=5
                          , return_train_score=True
                          , scoring=r2, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] max_depth=100, max_features=auto, n_estimators=100 ..............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=100, max_features=auto, n_estimators=100, total=   3.1s
[CV] max_depth=100, max_features=auto, n_estimators=100 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s


[CV]  max_depth=100, max_features=auto, n_estimators=100, total=   3.1s
[CV] max_depth=100, max_features=auto, n_estimators=100 ..............
[CV]  max_depth=100, max_features=auto, n_estimators=100, total=   3.2s
[CV] max_depth=100, max_features=auto, n_estimators=100 ..............
[CV]  max_depth=100, max_features=auto, n_estimators=100, total=   3.1s
[CV] max_depth=100, max_features=auto, n_estimators=100 ..............
[CV]  max_depth=100, max_features=auto, n_estimators=100, total=   3.1s
[CV] max_depth=100, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=100, max_features=auto, n_estimators=200, total=   6.2s
[CV] max_depth=100, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=100, max_features=auto, n_estimators=200, total=   6.3s
[CV] max_depth=100, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=100, max_features=auto, n_estimators=200, total=   6.2s
[CV] max_depth=100, max_features=auto, n_estimators=200 ..............

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  8.6min finished


RandomForestRegressor(max_depth=500, max_features='sqrt', n_estimators=500)
0.5101167690792761


In [21]:
print(grid_search.best_estimator_)

print(grid_search.best_score_)

RandomForestRegressor(max_depth=500, max_features='sqrt', n_estimators=500)
0.5101167690792761


In [26]:
predictions = (grid_search.predict(X_test))
print(f'Model scoring: {type(grid_search)}')
print(f"r2-score = {r2_score(y_test, predictions)}")
print(f"mse = {mean_squared_error(y_test, predictions)}")
print(f"rmse = {math.sqrt(mean_squared_error(y_test, predictions))}")
print()

Model scoring: <class 'sklearn.model_selection._search.GridSearchCV'>
r2-score = 0.5038411940731131
mse = 0.376123289275324
rmse = 0.6132889117498571



In [25]:
X_test = scaler.transform(X_test)

In [20]:
forest_reg.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [28]:
forest_reg.fit(X_train, y_train)

RandomForestRegressor()

In [29]:
predictions = (forest_reg.predict(X_test))
print(f'Model scoring: {type(grid_search)}')
print(f"r2-score = {r2_score(y_test, predictions)}")
print(f"mse = {mean_squared_error(y_test, predictions)}")
print(f"rmse = {math.sqrt(mean_squared_error(y_test, predictions))}")
print()

Model scoring: <class 'sklearn.model_selection._search.GridSearchCV'>
r2-score = 0.5081791385685663
mse = 0.372834822089348
rmse = 0.6106020161196227

